In [ ]:
!pip install kmapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install igraph plotly ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.1 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import networkx as nx
import kmapper as km
import matplotlib.pyplot as plt
from transformers import BertModel, BertTokenizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from kmapper.plotlyviz import plotlyviz
from sklearn.decomposition import PCA


In [ ]:
import pandas as pd

data_water = pd.read_parquet("hf://datasets/acmc/watermarked_c4_dataset/data/train-00000-of-00001.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data_water.head()

,text,timestamp,url,generated,model,results
0,IF we had a consumer confidence index for worl...,2019-04-21 08:44:12,https://www.csmonitor.com/1986/0506/aearl06.html,True,watermarking_facebook/opt-1.3b,"[{'confidence': 0.9999999999998954, 'generated..."
1,What are neutrophils and what do they do?\nNeu...,2019-04-22 22:36:18,http://lp.medicalnewstoday.com/articles/323982...,False,None,"[{'confidence': 0.8047903001943726, 'generated..."
2,It's like walking on hot coals.\nArtificial tu...,2019-04-19 10:50:37,https://www.nydailynews.com/new-york/parks-fak...,False,None,"[{'confidence': 0.8681516859117973, 'generated..."
3,Newsweek news director Cristina Silva will lea...,2019-04-20 14:57:18,https://www.thewrap.com/newsweek-turmoil-conti...,True,watermarking_facebook/opt-1.3b,"[{'confidence': 0.9999999999999994, 'generated..."
4,"Some 3,839 new vehicles were registered in Qat...",2019-04-25 20:12:18,https://www.gulf-times.com/story/561984/More-t...,False,None,"[{'confidence': 0.9903090626982858, 'generated..."


In [ ]:
# label_dict = {0:True, 1:False}
data_water['label'] = data_water['generated'].astype(int) #.map(label_dict)

In [ ]:
data_water.head()

,text,timestamp,url,generated,model,results,label
0,IF we had a consumer confidence index for worl...,2019-04-21 08:44:12,https://www.csmonitor.com/1986/0506/aearl06.html,True,watermarking_facebook/opt-1.3b,"[{'confidence': 0.9999999999998954, 'generated...",1
1,What are neutrophils and what do they do?\nNeu...,2019-04-22 22:36:18,http://lp.medicalnewstoday.com/articles/323982...,False,None,"[{'confidence': 0.8047903001943726, 'generated...",0
2,It's like walking on hot coals.\nArtificial tu...,2019-04-19 10:50:37,https://www.nydailynews.com/new-york/parks-fak...,False,None,"[{'confidence': 0.8681516859117973, 'generated...",0
3,Newsweek news director Cristina Silva will lea...,2019-04-20 14:57:18,https://www.thewrap.com/newsweek-turmoil-conti...,True,watermarking_facebook/opt-1.3b,"[{'confidence': 0.9999999999999994, 'generated...",1
4,"Some 3,839 new vehicles were registered in Qat...",2019-04-25 20:12:18,https://www.gulf-times.com/story/561984/More-t...,False,None,"[{'confidence': 0.9903090626982858, 'generated...",0


In [ ]:
water = data_water[data_water['generated'] == True][0:200]
nonwater = data_water[data_water['generated'] == False][0:200]

In [ ]:


# Load BERT model & tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)

def get_bert_attention(text):
    """Extracts BERT attention matrix for a given text"""
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)

    attention_matrices = torch.stack(outputs.attentions).mean(dim=0).squeeze().numpy()
    return np.mean(attention_matrices, axis=0)  # Averaging across all heads

def build_graph(attention_matrix, threshold=0.1):
    """Converts attention matrix into an undirected graph"""
    graph = nx.Graph()
    num_nodes = attention_matrix.shape[0]

    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if attention_matrix[i, j] > threshold:  # Only keep strong attention links
                graph.add_edge(i, j, weight=attention_matrix[i, j])

    return graph


# # With TSNE
# def extract_graph_features(graph):
#     """Extracts node features using t-SNE for dimensionality reduction"""
#     adjacency_matrix = nx.to_numpy_array(graph)

#     # Ensure perplexity is smaller than the number of nodes
#     perplexity = min(5, adjacency_matrix.shape[0] - 1)  # Use at most 5 or (n-1)

#     embedding = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(adjacency_matrix)
#     return embedding

# With PCA
def extract_graph_features(graph):
    """Extracts node features using PCA instead of t-SNE"""
    adjacency_matrix = nx.to_numpy_array(graph)

    # Reduce to 2D using PCA
    embedding = PCA(n_components=2).fit_transform(adjacency_matrix)
    return embedding



def mapper_visualization(embeddings, labels):
    """Applies Mapper to visualize the graph with node coloring based on labels"""
    mapper = km.KeplerMapper(verbose=1)

    # Scaling features to [0,1] for better visualization
    scaler = MinMaxScaler()
    scaled_embeddings = scaler.fit_transform(embeddings)

    # Create Mapper graph
    graph = mapper.map(
        scaled_embeddings,
        # lens=scaled_embeddings[:, 0],  # Use first dimension for filtering
        clusterer=km.cluster.DBSCAN(eps=0.3, min_samples=3),
        cover=km.Cover(n_cubes=10, perc_overlap=0.2)
    )

    # Visualize with plotly
    fig = plotlyviz(graph)#, color_values=labels)
    fig.show()


# def mapper_visualization(embeddings, labels):
#     """Applies Mapper to visualize the graph with node coloring based on labels"""
#     mapper = km.KeplerMapper(verbose=1)

#     # Scale embeddings for better visualization
#     scaler = MinMaxScaler()
#     scaled_embeddings = scaler.fit_transform(embeddings)

#     # Create Mapper graph
#     graph = mapper.map(
#         scaled_embeddings,
#         # lens=scaled_embeddings[:, 0],  # Use first dimension for filtering
#         clusterer=km.cluster.DBSCAN(eps=0.3, min_samples=3),
#         cover=km.Cover(n_cubes=10, perc_overlap=0.2)
#     )

#     # Ensure labels match the number of graph nodes
#     num_nodes = len(graph["nodes"])
#     adjusted_labels = np.random.choice(labels, num_nodes)  # Randomly assign existing labels

#     # Visualize
#     fig = plotlyviz(graph, color_values=adjusted_labels)
#     fig.show()




tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Water

texts = water['text']
labels = water['label']  # Example labels for coloring (change as needed)

# Process texts
attention_matrices = [get_bert_attention(text) for text in texts]
graphs = [build_graph(attn) for attn in attention_matrices]
embeddings = np.vstack([extract_graph_features(graph) for graph in graphs])

# Visualize with Mapper
mapper_visualization(embeddings, labels)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


KeplerMapper(verbose=1)
Mapping on data shaped (77239, 2) using lens shaped (77239, 2)

Creating 100 hypercubes.

Created 21 edges and 17 nodes in 0:00:27.615579.


In [ ]:
# Non-Water

texts = nonwater['text']
labels = nonwater['label']  # Example labels for coloring (change as needed)

# Process texts
attention_matrices = [get_bert_attention(text) for text in texts]
graphs = [build_graph(attn) for attn in attention_matrices]
embeddings = np.vstack([extract_graph_features(graph) for graph in graphs])

# Visualize with Mapper
mapper_visualization(embeddings, labels)

KeplerMapper(verbose=1)
Mapping on data shaped (76869, 2) using lens shaped (76869, 2)

Creating 100 hypercubes.

Created 24 edges and 18 nodes in 0:00:24.959699.


In [ ]:
# Example Usage
texts = ["This is a test sentence.", "Another example of text processing."]
labels = [0, 1]  # Example labels for coloring (change as needed)

# Process texts
attention_matrices = [get_bert_attention(text) for text in texts]
graphs = [build_graph(attn) for attn in attention_matrices]
embeddings = np.vstack([extract_graph_features(graph) for graph in graphs])

# Visualize with Mapper
mapper_visualization(embeddings, labels)

KeplerMapper(verbose=1)
Mapping on data shaped (16,) using lens shaped (16,)

Creating 10 hypercubes.

Created 0 edges and 1 nodes in 0:00:00.003779.


IndexError: index 3 is out of bounds for axis 0 with size 1